<a href="https://colab.research.google.com/github/thuylinh1204/DATN/blob/CrawlData/ExtractFull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

In [ ]:
!pip install jsonlines
!pip install python-docx 

In [ ]:
import docx
from google.colab import files
import jsonlines
import json
from copy import deepcopy

In [ ]:
number= []
i=0
while i<70:
  for j in range (0,20):
    number.append(str(i+1)+'.'+str(j))   
  i+=1

character= ['a)','b)','c)','d)','đ)','e).','f)','g)','h)','i)','k)',
       'l)','m)','n)','o)','p)','q)','s)','t)','u)','v)','x)','y)','z)',
       '(i)','(ii)','(iii)','(iv)','(v)','(vi)','(vii)','(viii)','(ix)','(x)',
       '(xi)','(xii)','(xiii)','(xiv)','(xv)','(xvi)','(xvii)','(xviii)','(xix)']

In [ ]:
part = 'Phần'
chapter = 'Chương'
section = 'Mục'
episode = 'Điểm'
episode_luat = 'Điều'
tags= []
tags =[*number, *character] 


In [ ]:
not_parts = ['Chương','Mục','Điều','Điểm', *tags]
not_chapters =  ['Phần','Mục','Điều','Điểm',*tags]
not_sections = ['Phần','Chương','Điều','Điểm',*tags]
not_episodes = ['Phần','Chương','Mục',*tags]      
not_tags = ['Phần','Chương','Điều','Mục','Điểm']

In [ ]:

def formatText (start_i,doc_len, doc,episode):
  array_part = []
  index_part = -1
  index_chapter = -1
  index_section = -1
  index_episode = -1
  index_context = -1
  phan = {}
  i= start_i
  n= doc_len
  while i<n-1:
    line = doc.paragraphs[i].text
    if line.startswith(part):
        if index_part >= 0:
            array_part.append(deepcopy(phan))
            index_chapter = -1
            index_section = -1
            index_episode = -1
            index_context = -1
            phan = {}
        index_part += 1 


        part_name = doc.paragraphs[i+1].text
        index_part +=1  
        i+=2
        next_line = doc.paragraphs[i].text
        while not next_line.startswith(tuple(not_parts)): 
          part_name += '  '+next_line
          i+=1
          next_line = doc.paragraphs[i].text
        phan['part_name'] = part_name
        phan['chapter'] = []
        continue
    
    elif line.startswith(chapter):
        if index_chapter >= 0:
            index_section = -1
            index_episode = -1
            index_context = -1
        index_chapter += 1

        chapter_name = doc.paragraphs[i+1].text
        i+=2
        next_line = doc.paragraphs[i].text
        while not next_line.startswith(tuple(not_chapters)): 
          chapter_name += '  '+next_line
          i+=1
          next_line = doc.paragraphs[i].text
        phan['chapter'].append({})
        phan['chapter'][index_chapter]['chapter_name'] = chapter_name
        phan['chapter'][index_chapter]['section'] = []
        continue

    elif line.startswith(section):
        if index_section >= 0:
            index_episode = -1
            index_context = -1
        index_section += 1

        section_name = doc.paragraphs[i+1].text
        i+=2
        next_line = doc.paragraphs[i].text
        while not next_line.startswith(tuple(not_sections)): 
          section_name += '  '+next_line
          i+=1
          next_line = doc.paragraphs[i].text

        phan['chapter'][index_chapter]['section'].append({})
        phan['chapter'][index_chapter]['section'][index_section]['section_name'] = section_name
        phan['chapter'][index_chapter]['section'][index_section]['episode'] = []
        continue

    elif line.startswith(episode):
          if index_episode >= 0:
              index_context = -1
          index_episode += 1
          episode_name = doc.paragraphs[i].text
          if index_chapter == -1:
              index_chapter += 1 
              phan['chapter'].append({})
              phan['chapter'][index_chapter]['section'] = []
              index_section+=1
              phan['chapter'][index_chapter]['section'].append({})
              phan['chapter'][index_chapter]['section'][index_section]['episode'] = []
          if index_section == -1:
              index_section+=1
              phan['chapter'][index_chapter]['section'].append({})
              phan['chapter'][index_chapter]['section'][index_section]['episode'] = []    
          i+=1
          
          phan['chapter'][index_chapter]['section'][index_section]['episode'].append({})
          phan['chapter'][index_chapter]['section'][index_section]['episode'][index_episode]['episode_name'] = episode_name
          phan['chapter'][index_chapter]['section'][index_section]['episode'][index_episode]['context'] = []
          continue

    elif (line.startswith(tag) for tag in tags):
        context= doc.paragraphs[i].text
        index_context+=1
        phan['chapter'][index_chapter]['section'][index_section]['episode'][index_episode]['context'].append(doc.paragraphs[i].text)
        i+=1
  return array_part       


In [ ]:
file_name = '/content/drive/MyDrive/PhapLuat/Thong_Tu_01_HN.docx' 
doc = docx.Document(file_name)

In [ ]:
n =len(doc.paragraphs)
n

In [ ]:
phan = formatText (12,n,doc,episode)

In [ ]:
time = []
for i in range(4, 9):
  time.append(doc.paragraphs[i].text)
time[0:5] = [' '.join(time[0:5])]
title = []
for i in range(2, 4):
  title.append(doc.paragraphs[i].text)

title[0:2] = [' '.join(title[0:2])]
base = []
for i in range(9,12):
  base.append(doc.paragraphs[i].text)

In [ ]:
rule = list()
processed = dict()
processed['title'] = title
processed['time'] = time
processed['base'] = base
processed['contents'] = phan

rule.append(processed)

In [ ]:
len(rule)

In [ ]:
file_name_luat = '/content/drive/MyDrive/PhapLuat/2022-11-Luat_So_huu_tri_tue-HN_lan_3.docx' 
doc_luat = docx.Document(file_name_luat)

In [ ]:
n_luat =len(doc.paragraphs)
n_luat

In [ ]:
array_part= formatText (14,n_luat,doc_luat,episode_luat)

In [ ]:
time_luat = []
for i in range(6, 10):
  time_luat.append(doc_luat.paragraphs[i].text)

time_luat[0:5] = [' '.join(time_luat[0:5])]
title_luat = []
for i in range(3, 5):
  title_luat.append(doc_luat.paragraphs[i].text)
title_luat[0:2] = [' '.join(title_luat[0:2])]
print(title_luat)
base_luat = []
for i in range(11, 12):
  base_luat.append(doc_luat.paragraphs[i].text)

In [ ]:
rule_luat = list()
processed_luat = dict()
processed_luat['title'] = title_luat
processed_luat['time'] = time_luat
processed_luat['base'] = base_luat
processed_luat['contents'] = array_part

rule_luat.append(processed_luat)

In [ ]:
rule_luat['contents']

In [ ]:
join_rule = [*rule, *rule_luat]

In [ ]:
len(join_rule)

In [ ]:

with jsonlines.open('/content/drive/MyDrive/PhapLuat/join_rule.jsonl', 'w') as writer:
    writer.write_all(join_rule)

In [ ]:

with open('/content/drive/MyDrive/PhapLuat/join_rule.jsonl', 'w', encoding='utf8') as f:
    for entry in join_rule:
        # print(entry, file=f)
        json.dump(entry, f, ensure_ascii=False)
        print('\n', file=f)